In [3]:
import requests
import json
import pprint
import pandas as pd
import csv
import re
from datetime import datetime
import time

# I. Lấy thông tin chung
- Lấy thông tin của tất cả các quán ăn thuộc 1 tỉnh/TP. Rộng hơn là thông tin tất cả quán ăn của nhiều tỉnh thành
- Với mỗi quán ăn, lấy những trường thông tin cơ bản như
    + name, addr, CityId, RestaurantId, RestaurantStatus <font color=green>// thông tin cơ bản</font>
    + AvgRating
    + TotalPictures, TotalSaves, TotalReviews, TotalFavourites, TotalViews, TotalCheckIns
    + CreatedOn <font color=green>//ngày shop dc tạo đầu tiên trên foody (trường này bị sao rồi, toàn mặc định)</font>
    + Latitude & Longitude <font color=green>//toạ độ</font>
    + ResUrlReviews <font color=green>//URL dẫn tới trang chứa bình luận của shop đó</font>


In [ ]:
## nháp
page=1
st=9
all_shops_API = requests.get(f"https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt=1&page={page}", 
                            headers={'X-Requested-With': 'XMLHttpRequest',
                                     'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC',
                                    })
data = all_shops_API.text 
dataJson = json.loads(data)

# print(len(dataJson['searchItems']))
dataJson['searchItems']

In [24]:
def get_single_page(page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text
    try:
        json_data = json.loads(data)
    except:
        return []
    if not json_data['searchItems']:
        return []
    shop_attribs = []
    
    for shop in json_data['searchItems']: 
        shop_attribs.append({'ReviewUrl' : shop['ReviewUrl'],
                            'Address': shop['Address']}) # bsung 
    
    return shop_attribs # [{},{},..]

def get_single_district(dist=1):
    '''get all shops in a single district'''
    page = 1
    all_shop = []
    empty = False

    for page in range(1, 101):
        if empty:
            break
        # print(f'page {page}')
        st = 1

        for st in range(1, 60):
            # print(f'  st = {st}')
            time.sleep(1)
            one_page = get_single_page(page, st, dist) 
            
            if not one_page: # check empty
                empty = True
                break
            all_shop.extend(one_page)
    return all_shop # [{},{},..]                       

def get_single_city(city_id='217', start_id=1):
    all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
    all_city_shops = []

    for dist in all_dist[start_id:]:
        all_city_shops.extend(get_single_district(dist))
    return all_city_shops

In [25]:
def save_district_to_file(dist):
    shops_in_a_district = get_single_district(dist)

    df = pd.DataFrame.from_dict(shops_in_a_district) 
    df.dropna().drop_duplicates().to_csv(f'q{dist}.csv', index = False, header=True, encoding='utf-8')

all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
for dist in all_dist[2:]:
    save_district_to_file(dist)
    print(f'save district {dist} successfully')

save district 5 successfully


In [ ]:
shops_in_a_district = get_single_district(dist=4)
print(len(shops_in_a_district))

df = pd.DataFrame.from_dict(shops_in_a_district) 
print(f'lọc trùng:', df.drop_duplicates().shape)
df.drop_duplicates().to_csv(r'q2_1.csv', index = False, header=True, encoding='utf-8')

In [19]:
# merge 
df = pd.read_csv('q2.csv')
df.drop_duplicates().to_csv(r'q2.csv', index = False, header=True, encoding='utf-8')
df.drop_duplicates().shape

(1580, 2)

# II. Lấy thông tin cụ thể từng quán ăn
Thông tin lần này chia thành 2 loại:
- Thông tin thực đơn (Get delivery dishes)
    - dish_type_name <font color=green>//Loại đồ ăn: 1 loại đồ ăn gồm nhiều món ăn, VD: bún thì có bún cá, bún chả</font>
    - total_like <font color=green>//tổng lượt like loại đồ ăn đó</font>
    - is_available <font color=green>//loại đồ ăn đó còn món ko</font>
    - dish_name <font color=green>//món ăn thuộc loại đồ ăn đó. VD: bún cá thập cẩm</font>
    - dish_price <font color=green>//giá 1 món ăn thuộc 1 loại đồ ăn </font>
    - options: <font color=green>// có option nào cho món ăn đó. VD: có option chọn thêm topping như rau, cua,... </font>
        [
        option_1: name, price;
        option_2: name, price;
        ....
        ]
    - option_min_select, option_max_select

- Thông tin chi tiết quán ăn (Get detail):
    - phone 
	- district_id
	- category <font color=green>//loại hình quán ăn?</font>
	- cuisines <font color=green>//quán ăn thuộc phân khúc món nào? món Việt, Âu,...?</font>
	- service_fee <font color=green>//fí dịch vụ</font>
	- avg_price <font color=green>// chắc là giá trung bình của quán ăn.</font>
	- min_order_value <font color=green>//fải mua tối thiểu bnhiêu tiền?</font>
	- promotion <font color=green>// trong này nhiều trường nhỏ, tự nghiên cứu thêm</font>
	- min_charge <font color=green>//fí fụ thu nhỏ nhất</font>
	- week_days, operating <font color=green>//thời gian mở cửa</font>
	- min_shipping_fee <font color=green>// có thể là giá ship thấp nhất</font>
	- is_foody_delivery <font color=green>// có thể là shop này có phải là shop bán mang về hay không</font>
	- price_range <font color=green>// khoảng giá min – max của quán.</font>

---

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
def hi():
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={restaurant_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )

In [ ]:
def get_delivery_dishes(restaurant_id):
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={restaurant_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )

        data = delivery_dish_API.text
        dataJson = json.loads(data)
        menu_infos = dataJson['reply']['menu_infos']

        menu_df = pd.DataFrame(columns=['restaurant_id', 'dish_type_id', 'dish_type_name'])
        menu_dish_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
        menu_dish_option_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
        menu_dish_option_item_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

        for menu_info in menu_infos:
                menu_df.loc[len(menu_df.index)] = [restaurant_id, menu_info['dish_type_id'], menu_info['dish_type_name']]
                for dish in menu_info['dishes']:
                        menu_dish_df.loc[len(menu_dish_df.index)] = [
                                menu_info['dish_type_id'],
                                dish['id'],
                                dish['name'],
                                dish['description'],
                                dish['price']['value'],
                                dish['total_like'],
                                dish['is_available']
                        ]
                        for option in dish['options']:
                                menu_dish_option_df.loc[len(menu_dish_option_df.index)] = [
                                        menu_info['dish_type_id'],
                                        dish['id'],
                                        option['id'],
                                        option['option_items']['min_select'],
                                        option['option_items']['max_select']
                                ]
                                for item in option['option_items']['items']:
                                        menu_dish_option_item_df.loc[len(menu_dish_option_item_df.index)] = [
                                                menu_info['dish_type_id'],
                                                dish['id'],
                                                option['id'],
                                                item['id'],
                                                item['name'],
                                                item['price']['value']
                                        ]
        return menu_df, menu_dish_df, menu_dish_option_df, menu_dish_option_item_df

In [ ]:
all_shop_menu = pd.DataFrame()

# get menu of all shops
for res_id in all_restaurant_ids:
    menu_df, menu_dish_df, menu_dish_option_df, menu_dish_option_item_df = get_delivery_dishes(res_id)
    all_shop_menu.append(menu_df)

all_shop_menu.to_csv('menu.csv', sep=',', encoding='utf-8', index=False)
# menu_dish_df.to_csv('menu.dish.csv', sep=',', encoding='utf-8', index=False)
# menu_dish_option_df.to_csv('menu.dish.option.csv', sep=',', encoding='utf-8', index=False)
# menu_dish_option_item_df.to_csv('menu.dish.option.item.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
# TEST: có thể thay request_id = 1031704 để thử
# ...

- get detail of ONE shop

In [ ]:
def get_detail(restaurant_id):
        delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={restaurant_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        data = delivery_dish_API.text
        dataJson = json.loads(data)
        shop = dataJson['reply']['delivery_detail']

        shop_df = pd.DataFrame(columns=['shop_id', 'name', 'short_description', 'city_id', 'district_id', 'address', 
                                'rating_total_review', 'rating_avg', 'rating_app_link',  
                                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'min_shipping_fee', 'min_price', 'max_price', 
                                'is_foody_delivery', 'week_days', 'start_time', 'end_time'])

        shop_phone_df = pd.DataFrame(columns=['shop_id', 'phone'])
        shop_category_df = pd.DataFrame(columns=['shop_id', 'category'])
        shop_cuisine_df = pd.DataFrame(columns=['shop_id', 'cuisine'])
        shop_promotion_df = pd.DataFrame(columns=['shop_id', 'promo_code', 'discount_amount', 'min_order_amount', 'max_discount_amount', 'apply_order','max_usage_time', 'expired'])

        shop_df.loc[len(shop_df.index)] = [
                shop['id'], shop['name'], shop['short_description'], shop['city_id'], shop['district_id'], shop['address'],
                shop['rating']['total_review'], shop['rating']['avg'], shop['rating']['app_link'],
                shop['delivery']['service_fee']['value'], shop['delivery']['avg_price']['value'], shop['delivery']['min_order_value']['value'],
                shop['delivery']['min_charge'], shop['delivery']['shipping_fee']['minimum_fee'],shop['price_range']['min_price'],
                shop['price_range']['max_price'],shop['delivery']['is_foody_delivery'],
                len(shop['delivery']['time']['week_days']), shop['delivery']['time']['week_days'][0]['start_time'], shop['delivery']['time']['week_days'][0]['end_time']
        ]

        for phone in shop['phones']:
                shop_phone_df.loc[len(shop_phone_df.index)] = [
                        shop['id'],
                        phone
                ]

        for category in shop['categories']:
                shop_category_df.loc[len(shop_category_df.index)] = [
                        shop['id'],
                        category
                ]                     

        for cuisine in shop['cuisines']:
                shop_cuisine_df.loc[len(shop_cuisine_df.index)] = [
                        shop['id'],
                        cuisine
                ]    

        for promotion in shop['delivery']['promotions']:
                shop_promotion_df.loc[len(shop_promotion_df.index)] = [
                        shop['id'], promotion['promo_code'], promotion['discount_amount'], promotion['min_order_amount'], 
                        promotion['max_discount_amount'], promotion['apply_order'], promotion['user_condition']['limit_per_user']['max_usage_time'], promotion['expired']
                ]

        shop_df.to_csv('shop.csv', sep=',', encoding='utf-8')
        shop_phone_df.to_csv('shop.phone.csv', sep=',', encoding='utf-8')
        shop_category_df.to_csv('shop.category.csv', sep=',', encoding='utf-8')
        shop_cuisine_df.to_csv('shop.cuisine.csv', sep=',', encoding='utf-8')
        shop_promotion_df.to_csv('shop.promotion.csv', sep=',', encoding='utf-8')